<center><h1>SEOUL BIKE TRIP DURATION PREDICTION</h1></center>
<center><img src="https://storage.googleapis.com/kaggle-datasets-images/1182717/1978889/22959fc75a57e2249067c80aba6b338c/dataset-cover.jpg?t=2021-02-26-19-04-48" /><center>

<b>Problem Statement: </b>The duration of a trip is the most crucial measure in all modes of transportation. Hence, it is crucial to predict the trip time precisely for the advancement of Intelligent Transport Systems (ITS) and traveler information systems. The prediction can be done using the Seoul Bike data and weather data. We need to use this combination of Seoul Bike data and weather data to do the trip duration prediction.​

In [2]:
from IPython.display import YouTubeVideo
YouTubeVideo('https://www.youtube.com/watch?v=R-k78IVzYy0&list=PLsR_0x6BuM-EWzaEqBhrHh79SnJSa-Sx6&index=2&ab_channel=cloudyML',width=700, height=400)

## ABOUT DATASET 
- <b>Dataset File Name:</b> `for_modeling.csv`
- <b>Dataset Shape:</b> (9601139, 26)
- <b>Columns in the dataset:</b>
    - Unnamed: 0
    - Duration 
    - Distance
    - PLong
    - PLatd
    - DLong
    - DLatd
    - Haversine
    - Pmonth
    - Pday
    - Phour
    - Pmin
    - PDweek
    - Dmonth
    - Dday
    - Dhour
    - Dmin
    - DDweek
    - Temp
    - Precip
    - Wind
    - Humid
    - Solar
    - Snow
    - GroundTemp
    - Dust

### Importing required libraries

In [4]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import random
import warnings

np.random.seed(24)
random.seed(24)
warnings.filterwarnings("ignore")
from matplotlib.pyplot import figure
figure(num=None, figsize=(15, 15), dpi=80, facecolor='w', edgecolor='k')

In [4]:
# import dataset
# from google.colab import drive
# drive.mount('/content/drive')

### Importing the dataset

In [5]:
# Read csv using pandas
data = pd.read_csv('../input/seoul-bike-trip-duration-prediction/For_modeling.csv')

In [6]:
# check the shape of the data
data.shape

In [6]:
# sample 100000 rows of data
sample = data.sample(100000)

In [8]:
# check the shape
sample.shape

In [9]:
# check the top 5 rows
sample.head()

## Exploring the Data
### Identifying the number of features or columns

In [10]:
# Check all the columns in the dataset
sample.columns

### Drop irrelevant columns from dataset

In [7]:
# Dropping irrelevant column from the dataset i.e Unnamed: 0
sample.drop(['Unnamed: 0'], axis = 1, inplace = True)

### Knows more about the data in the columns like data type it contains and total samples of each

In [12]:
# Check which columns are having categorical, numerical or boolean values
sample.info()

**After checking the Dtypes of all the columns**

**object - String values**

**float64 - Numerical values**

**Observation: There are no String values so there are no categorical data**

### Know more mathematical relations of the dataset like count, min, max values, standarad deviation values, mean and different percentile values

In [13]:
# For more information on the dataset like the total count in all the columns
# min, max values and more information of the respective columns 
sample.describe()

### Get the total number of samples in the dataset using the len() function

In [14]:
# check length of dataset
len(sample)

### Get unique values

In [15]:
# check for unique values in dataset
sample.nunique()

### Counting the total number of missing value

In [16]:
# Check for missing values in all the columnns of the dataset
sample.isnull().sum()

There is no missing values in this dataset

## Exploratory Data Analysis (EDA)

### CORRELATION MATRIX


Why ?

**A correlation matrix is a table showing correlation coefficients between variables**

There are three broad reasons for computing a correlation matrix:
1. To summarize a large amount of data where the goal is to see patterns. In our example above, the observable pattern is that all the variables highly correlate with each other.
2. To input into other analyses. For example, people commonly use correlation matrixes as inputs for exploratory factor analysis, confirmatory factor analysis, structural equation models, and linear regression when excluding missing values pairwise.
3. As a diagnostic when checking other analyses. For example, with linear regression, a high amount of correlations suggests that the linear regression estimates will be unreliable.


In [17]:
#correlation matrix Using pandas
corr = sample.corr()
corr.style.background_gradient(cmap='coolwarm')

In [18]:
# correlation matrix Using seaborn
plt.figure(figsize=(25,25))
sns.heatmap(sample.corr(), annot = True)
plt.show()

**Observation from above correlation matrix**
1. distance and duration are strongly related
2. haversine and duration and correlated with each other
3. gound temperature and solar radiation are correlated
4. temperature and solar radiations are correlated
5. phour and dhour are not related to humidity

## SCATTER PLOT

1. A scatter plot is a type of plot using Cartesian coordinates to display values for typically two variables for a set of data.

2. The data are displayed as a collection of points, each having the value of one variable determining the position on the horizontal axis and the value of the other variable determining the position on the vertical axis.

3. Scatter plot's are used to observe and show relationships between two numeric variables.

In [19]:
# Scatter plot using matplotlib 
# create function for ploting scatterplot between two columns of dataset
def draw_scatter_plot(col1, col2):
    plt.scatter(x = col1, y = col2)
    
# Loop through numerical data list and use function to scatter plot between two columns
for i in sample.columns:
    for j in sample.columns:
        if j!= i:
            draw_scatter_plot(sample[i], sample[j])
            plt.xlabel(i)
            plt.ylabel(j)
            plt.show()

PLong and DLong are correlated

PLatd and DLatd are correlated

Pmonth and Dmonth are correlated

## HISTOGRAM

1. A histogram is an approximate representation of the distribution of numerical data.

2. To construct a histogram, the first step is to "bin" (or "bucket") the range of values—that is, divide the entire range of values into a series of intervals—and then count how many values fall into each interval.

3. The words used to describe the patterns in a histogram are: "symmetric", "skewed left" or "right", "unimodal", "bimodal" or "multimodal".

In [20]:
#Histogram Using pandas 
sample.hist(figsize=(15,20))

Observation -
2. data distribution of duration, distance, haversine, precipation, solar radiation, snow and dust are skewed right.
3. data distribution of humidity is symmetrical
4. data distribution of Pmin and Dmin is right skewed.
4. data distribution of Phour and Dhour is same.
4. data distribution of pday and Dday is same.
4. data distribution of Pmonth and Dmonth is same

## VIF - Variance inflation factor
1. The variance inflation factor (VIF) quantifies the extent of correlation between one predictor and the other predictors in a model.
2. It is used for diagnosing collinearity/multicollinearity.
3. Higher values signify that it is difficult to impossible to assess accurately the contribution of predictors to a model.

In [21]:
# import statsmodle library for vif 
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [22]:
sample.columns

In [23]:
# creating a dataframe of just numerical values
X = sample.drop(['Duration'], axis = 1)
# target values
y = sample['Duration']
# numerical values column names
X.columns



In [24]:
X

In [25]:
# droping rows with empty cells
X.dropna(axis = 0, how = 'any', inplace = True)
X.reset_index(drop = True, inplace = True)

In [26]:
X

In [27]:
from sklearn.metrics import r2_score

In [28]:
# Calculating VIF for each feature.
# vif_df = pd.DataFrame()
# vif_df['feature'] = X.columns
# vif_df['feature'] = [variance_inflation_factor(X.values, i)
#                           for i in range(len(X.columns))]

for i in range(len(X.columns)):
#     print("R Square value of", X.columns[i], "columns is", r2_score(X[X.columns[~i]].values, X[X.columns[i]].values))
    print("VIF for", X.columns[i] ,"column is ", round(variance_inflation_factor(X.values, i), 2))

there is colinearity/multicolinearity between variables as the VIF value is almost upto 2.5

Distance', 'PLong', 'PLatd', 'DLong','DLatd',  'Phour',  'PDweek', 'Dday', 'Dhour', 'Dmin', 'DDweek', 'Temp', 'Precip', 'Humid', 'Solar', 'Snow', 'GroundTemp',  have colinearity with all the variables.

## Box Plot

A boxplot is a standardized way of displaying the dataset based on a five-number summary:

    1. Minimum (Q0 or 0th percentile): the lowest data point excluding any outliers.

    2. Maximum (Q4 or 100th percentile): the largest data point excluding any outliers.

    3. Median (Q2 or 50th percentile): the middle value of the dataset.

    4. First quartile (Q1 or 25th percentile): also known as the lower quartile qn(0.25), is the median of the lower half of the dataset.

    5. Third quartile (Q3 or 75th percentile): also known as the upper quartile qn(0.75), is the median of the upper half of the dataset

In [29]:
# Perform boxplot on Duration column of the dataset
y.plot.box(grid='True')

from above box plot graph:

duration

25% of duration have value between range 0 to 8

25% of duration have value between range 8 to 18

25% of duration have value between range 18 to 35

25% of duration have value between range 35 to 77

The mean duration is around 18

**Mostly the duration are on the lower side of the spectrum. Means small duration trips are the common trips**


In [30]:
# Perform boxplot on Distance column of the dataset
X['Distance'].plot.box(grid = True)

from above box plot graph:

distance

25% of distance have value between range 0 to 1000

25% of distance have value between range 1000 to 2500

25% of distance have value between range 2500 to 4900

25% of distance have value between range 4900 to 9900

The mean distance is around 2500

 Mostly the distance are on the lower side of the spectrum. Means small distance trips are the common trips



In [31]:
# Perform boxplot on PLong column of the dataset
X['PLong'].plot.box(grid = True)

from above box plot graph:

PLong

25% of PLong have value between range 37.44 to 37.52

25% of PLong have value between range 37.52 to 37.559

25% of PLong have value between range 37.559 to 37.58

25% of PLong have value between range 37.58 to 37.66

The mean PLong is around 37.559

In [32]:
# Perform boxplot on PLatd column of the dataset
X['PLatd'].plot.box(grid = True)

from above box plot graph:

PLatd

25% of PLatd have value between range 126.80 to 162.92

25% of PLatd have value between range 126.92 to 126.99

25% of PLatd have value between range 126.99 to 127.07

25% of PLatd have value between range 127.07 to 127.18

The mean PLatd is around 126.99

In [33]:
# Perform boxplot on Dlong column of the dataset
X['DLong'].plot.box(grid = True)

from above box plot graph:

DLong

25% of DLong have value between range 37.43 to 37.52

25% of DLong have value between range 37.52 to 37.545

25% of DLong have value between range 37.545 to 37.575

25% of DLong have value between range 37.575 to 37.665

The mean DLong is around 37.545

In [34]:
# Perform boxplot on DLatd column of the dataset
X['DLatd'].plot.box(grid = True)

from above box plot graph:

DLatd

25% of DLatd have value between range 126.80 to 126.92

25% of DLatd have value between range 126.92 to 126.995

25% of DLatd have value between range 126.995 to 127.055

25% of DLatd have value between range 127.055 to 127.175

The mean DLatd is around 126.995

In [35]:
# Perform boxplot on Haversine column of the dataset
X['Haversine'].plot.box(grid = True)

from above box plot graph:

haversine

25% of haversine have value between range 0 to 0.25

25% of haversine have value between range 0.25 to 1

25% of haversine have value between range 1 to 2.5

25% of haversine have value between range 2.5 to 5

The mean haversine is around 1

Observation: Mostly the haversine are on the lower side of the spectrum. Means small distance trips are the common trips

In [36]:
# Perform boxplot on pmonth column of the dataset
X['Pmonth'].plot.box(grid = True)

from above box plot graph:

Pmonth

25% of Pmonth have value between range 1 to 6

25% of Pmonth have value between range 6 to 8

25% of Pmonth have value between range 8 to 10

25% of Pmonth have value between range 10 to 12

The mean Pmonth is around 8

more pickups are in the month which are at the end of the year

In [157]:
# Perform boxplot on Pday column of the dataset
X['Pday'].plot.box(grid = True)

from above box plot graph:

Pday

25% of Pday have value between range 1 to 8

25% of Pday have value between range 8 to 16

25% of Pday have value between range 16 to 23

25% of Pday have value between range 23 to 31

The mean Pday is around 16

pickups are almost same on every day of month

In [158]:
# Perform boxplot on Phour column of the dataset
X['Phour'].plot.box(grid = True)

from above box plot graph:

Phour

25% of Phour have value between range 0 to 10

25% of Phour have value between range 10 to 16

25% of Phour have value between range 16 to 19

25% of Phour have value between range 19 to 24

The mean Phour is around 16

more pickups are towards the end of the day

In [159]:
# Perform boxplot on Pmin column of the dataset
X['Pmin'].plot.box(grid = True)

from above box plot graph:

Pmin

25% of Pmin have value between range 0 to 14

25% of Pmin have value between range 14 to 29

25% of Pmin have value between range 29 to 44

25% of Pmin have value between range 44 to 60

The mean Pmin is around 29

In [160]:
# Perform boxplot on PDweek column of the dataset 
X['PDweek'].plot.box(grid = True)

from above box plot graph:

PDweek

25% of PDweek have value between range 0 to 1

25% of PDweek have value between range 1 to 3

25% of PDweek have value between range 3 to 5

25% of PDweek have value between range 3 to 6

The mean PDweek is around 3

equal pickups throughout the week

In [161]:
# Perform boxplot on Dmonth column of the dataset
X['Dmonth'].plot.box(grid = True)

from above box plot graph:

Dmonth

25% of Dmonth have value between range 1 to 6

25% of Dmonth have value between range 6 to 8

25% of Dmonth have value between range 8 to 10

25% of Dmonth have value between range 10 to 12

The mean Dmonth is around 8

more drop offs are towords the end months of the year

In [162]:
# Perform boxplot on Dday column of the dataset 
X['Dday'].plot.box(grid = True)

from above box plot graph:

Dday

25% of Dday have value between range 1 to 8

25% of Dday have value between range 8 to 16

25% of Dday have value between range 16 to 23

25% of Dday have value between range 23 to 31

The mean Dday is around 16

drops the equal throughout all the days of month

In [163]:
# Perform boxplot on Dhour column of the dataset
X['Dhour'].plot.box(grid = True)

from above box plot graph:

Dhour

25% of Dhour have value between range 0 to 10

25% of Dhour have value between range 10 to 16

25% of Dhour have value between range 16 to 20

25% of Dhour have value between range 20 to 24

The mean Dhour is around 16

drops offs are more towards the end hours of the day

In [164]:
# Perform boxplot on Dmin column of the dataset
X['Dmin'].plot.box(grid = True)

from above box plot graph:

Dmin

25% of Dmin have value between range 0 to 15

25% of Dmin have value between range 15 to 30

25% of Dmin have value between range 30 to 45

25% of Dmin have value between range 45 to 60

The mean Dmin is around 30

In [165]:
# Perform boxplot on DDweek column of the dataset
X['DDweek'].plot.box(grid = True)

from above box plot graph:

DDweek

25% of DDweek have value between range 0 to 1

25% of DDweek have value between range 1 to 3

25% of DDweek have value between range 3 to 5

25% of DDweek have value between range 5 to 6

The mean DDweek is around 3

drops are equal throughout the week

In [166]:
# Perform boxplot on Temp column of the dataset
X['Temp'].plot.box(grid = True)

from above box plot graph:

temperature

25% of temperature have value between range -8 to 2

25% of temperature have value between range 2 to 20

25% of temperature have value between range 20 to 26

25% of temperature have value between range 26 to 40

The mean temperature is around 20

trips are planned in slightly high temperature

In [167]:
# Perform boxplot on Precip column of the dataset
# Using pandas 
X['Precip'].plot.box(grid = True)

from above box plot graph:

precip

25% of precip have value between range 0 to 0

25% of precip have value between range 0 to 0

25% of precip have value between range 0 to 0

25% of precip have value between range 0 to 0

The mean precip is around 0

more trips are planned when there is no rain or snow fall

In [168]:
# Perform boxplot on wind column of the dataset
X['Wind'].plot.box(grid = True)

from above box plot graph:

wind

25% of wind have value between range 0 to 1.1

25% of wind have value between range 1.1 to 1.8

25% of wind have value between range 1.8 to 2.5


25% of wind have value between range 2.5 to 4.2

The mean wind is around 1.8

more trips are during less windy wheather

In [169]:
# Perform boxplot on humid column of the dataset
X['Humid'].plot.box(grid = True)

from above box plot graph:

humid

25% of humid have value between range 10 to 41

25% of humid have value between range 41 to 56

25% of humid have value between range 56 to 64

25% of humid have value between range 64 to 99

The mean humid is around 56

In [170]:
# Perform boxplot on solar column of the dataset 
X['Solar'].plot.box(grid = True)

from above box plot graph:

solar

25% of solar have value between range 0 to 0

25% of solar have value between range 0 to 0.25

25% of solar have value between range 0.25 to 1.25

25% of solar have value between range 1.25 to 3.1

The mean solar is around 0.25

In [171]:
# Perform boxplot on snow column of the dataset 
X['Snow'].plot.box(grid = True)

from above box plot graph:

snow

25% of snow have value between range 0 to 0

25% of snow have value between range 0 to 0

25% of snow have value between range 0 to 0

25% of snow have value between range 0 to 0

The mean snow is around 0

more trips are during when there is no snow fall

In [172]:
# Perform boxplot on GroundTemp column of the dataset
X['GroundTemp'].plot.box(grid = True)

from above box plot graph:

groundtemperature

25% of groundtemperature have value between range -12 to 11

25% of groundtemperature have value between range 11 to 21

25% of groundtemperature have value between range 21 to 29

25% of groundtemperature have value between range 29 to 52

The mean groundtemperature is around 21

In [173]:
# Perform boxplot on dust column of the dataset
X['Dust'].plot.box(grid = True)

from above box plot graph:

dust

25% of dust have value between range 0 to 20

25% of dust have value between range 20 to 30

25% of dust have value between range 30 to 45

25% of dust have value between range 45 to 80

The mean dust is around 30

Observation: Mostly the dust are on the lower side of the spectrum. Means people preffer to travel in less dust concentration

## POINT PLOT


1. A point plot uses scatter plot glyphs to visualize features like point estimates and confidence intervals.

2. A point plot uses scatter plot points to represent the central tendency of numeric data.

3. These plots make use of error bars to indicate any uncertainty around the numeric

In [174]:
# Perform point plot between Phour and Duration values 
sns.pointplot(x = 'Phour', y = 'Duration', data = sample)

duration is more during morning and evening

after 8 duration is increasing and after 20 its again decreasing

In [175]:
# Perform point plot between distance and Duration values 
sns.pointplot(x = 'Distance', y = 'Duration', data = sample)

duration and distance have a positive correlation between them



as duration increases distance also increases

In [176]:
# Perform point plot between PLong and Duration values 
sns.pointplot(x = 'PLong', y = 'Duration', data = sample)

as duration  increases  pickup longitude decreases

In [177]:
# Perform point plot between PLatd and Duration values 
sns.pointplot(x = 'PLatd', y = 'Duration', data = sample)

duration and PLatd has negative correlation between them.

as the value of duration increases value of PLatd decreases

In [178]:
# Perform point plot between DLong and Duration values 
sns.pointplot(x = 'DLong', y = 'Duration', data = sample)

duration and DLong has negative correlation between them.

as the value of duration increases value of DLong decreases

In [179]:
# Perform point plot between haversine and Duration values 
sns.pointplot(x = 'Haversine', y = 'Duration', data = sample)

duration and haversine are correlated 

they have positive correlation between them

In [180]:
# Perform point plot between Pmonth and Duration values 
sns.pointplot(x = 'Pmonth', y = 'Duration', data = sample)

pmonth and duration are positively correlated

as the value of duration increases value of Pmonth also increases






In [181]:
# Perform point plot between Pday and Duration values 
sns.pointplot(x = 'Pday', y = 'Duration', data = sample)

most of the points are between 14.0 and 15.0

very few are above 15,0 and below 14.0

In [182]:
# Perform point plot between PDweek and Duration values 
sns.pointplot(x = 'PDweek', y = 'Duration', data = sample)

Duration has a positive correlation with PDweek

as the value of duration increasing value of PDweek is also increasing

In [183]:
# Perform point plot between Dmonth and Duration values 
sns.pointplot(x = 'Dmonth', y = 'Duration', data = sample)

all values are between 7.5 and 7.8 

there is no relation between Dmonth and Duration

In [184]:
# Perform point plot between Dhour and Duration values 
sns.pointplot(x = 'Dhour', y = 'Duration', data = sample)

there is a positive relation between duration and Dhour 

as the value of duration increases value of Dhour also increases

In [185]:
# Perform point plot between DDweek and Duration values 
sns.pointplot(x = 'DDweek', y = 'Duration', data = sample)

there is increase in DDweek when there is increase in duration

they are correlated to each other

In [186]:
# Perform point plot between Temp and Duration values 
sns.pointplot(x = 'Temp', y = 'Duration', data = sample)

duration and templerature have a positive relation between them


In [187]:
# Perform point plot between Precip and Duration values 
sns.pointplot(x = 'Precip', y = 'Duration', data = sample)

most of the values are between 0.0 and 0.01

very few values are above 0.01

In [188]:
# Perform point plot between wind and Duration values 
sns.pointplot(x = 'Wind', y = 'Duration', data = sample)

duration and wind are correlated 

as the value of duration increases value of wind is also increasing

In [189]:
# Perform point plot between humid and Duration values 
sns.pointplot(x = 'Humid', y = 'Duration', data = sample)

duration and humidity are negatively correlated 

as the value of duration is increasing value of humidity decreases

In [190]:
# Perform point plot between solar and Duration values 
sns.pointplot(x = 'Solar', y = 'Duration', data = sample)

as the value of duration is increasing value of solar radiations is also increasing

they are correlated to each other

In [191]:
# Perform point plot between snow and Duration values 
sns.pointplot(x = 'Snow', y = 'Duration', data = sample)

snow and duration are correlated 

they have negative relation between them

In [192]:
# Perform point plot between GroundTemp and Duration values 
sns.pointplot(x = 'GroundTemp', y = 'Duration', data = sample)

ground temperature and duration are correlated to each other.

as the value of duration increases value of ground temperature also increases

In [193]:
# Perform point plot between dust and Duration values 
sns.pointplot(x = 'Dust', y = 'Duration', data = sample)

dust and duration are correlated to each other.

as the value of duration increases value of dust also increases

In [194]:
# Perform point plot between precip and distance values 
sns.pointplot(x = 'Precip', y = 'Distance', data = sample)

all the values are around 0.00

very few are above 0.00

In [195]:
# Perform point plot between PLong and Dlong values 
sns.pointplot(x = 'PLong', y = 'DLong', data = sample)

Plong and Dlong are corelated

In [196]:
# Perform point plot between PLong and haversine values 
sns.pointplot(x = 'PLong', y = 'Haversine', data = sample)

all the values are between 1 and 3

very few are above 3 and below 1

In [197]:
# Perform point plot between PLong and Pday values 
sns.pointplot(x = 'PLong', y = 'Pday', data = sample)

all the values are between 12.5 to 17.5

very few are above 17.5 and below 12.5

In [198]:
# Perform point plot between PLong and Pmin values 
sns.pointplot(x = 'PLong', y = 'Pmin', data = sample)

all the values are between 25 to 35

very few are above and below of this range

In [199]:
# Perform point plot between PLong and PDweek values 
sns.pointplot(x = 'PLong', y = 'PDweek', data = sample)

all the values are between 2 to 4

very few are above and below of this range

In [200]:
# Perform point plot between PLong and Dday values 
sns.pointplot(x = 'PLong', y = 'Dday', data = sample)

all the values are between 12 to 17

very few are above and below of this range

In [201]:
# Perform point plot between PLong and Precip values 
sns.pointplot(x = 'PLong', y = 'Precip', data = sample)

all values are around 0.00 

very few are above 0.00

In [202]:
# Perform point plot between PLong and Humind values 
sns.pointplot(x = 'PLong', y = 'Humid', data = sample)

as the distribution value of PLong is increasing the value of distribution of humidity is same

In [203]:
# Perform point plot between PLatd and Dday values 
sns.pointplot(x = 'PLatd', y = 'Dday', data = sample)

all valies are around 15

very few are above and below of 15

## COUNT PLOT

1. A countplot is kind of like a histogram or a bar graph for some categorical area.

2. It simply shows the number of occurrences of an item based on a certain type of category.


In [206]:
# Perform the countplot on the pickup hour data
sns.countplot(x = 'Phour', data = sample)

more pickups are on morning and evening 

In [208]:
# Perform the countplot on the drop hour data
sns.countplot(x = 'Dhour', data = sample)

Observation:  Dropoff hour are high at the office hours both at morning and evening



In [209]:
# Perform the countplot on the duration data
sns.countplot(x = 'Duration', data = sample)

distribution of duration is skewed left side



short duration trips are common

In [210]:
# Perform the countplot on the distance data
sns.countplot(x = 'Distance', data = sample)

distribution of data is skewed towards left 

short distance trips are common

In [211]:
# Perform the countplot on the PLong data
sns.countplot(x = 'PLong', data = sample)


distribution of data is multimodal

In [214]:
# Perform the countplot on the Platd data

sns.countplot(x = 'PLatd', data = sample)


distribution of data is multimodal

In [216]:
# Perform the countplot on the Dlong data
sns.countplot(x = 'DLong', data = sample)


distribution of data is multimodal

In [217]:
# Perform the countplot on the Dlatd data
sns.countplot(x = 'DLatd', data = sample)


distribution of data is multimodal

In [218]:
# Perform the countplot on the Pmonth data
sns.countplot(x = 'Pmonth', data = sample)


more picks are on end of the year

In [219]:
# Perform the countplot on the Pday data
sns.countplot(x = 'Pday', data = sample)


distribution of data is multimodal

pickups are  almost similar on every day of a month

In [220]:
# Perform the countplot on the pmin data
sns.countplot(x = 'Pmin', data = sample)


data is distributed equally

In [221]:
# Perform the countplot on the PDweek data
sns.countplot(x = 'PDweek', data = sample)


data is distributed equally

In [222]:
# Perform the countplot on the Dday data
sns.countplot(x = 'Dday', data = sample)


distribution of data is similar

drop-offs are almost similar on every day of a month

In [223]:
# Perform the countplot on the Dmin data
sns.countplot(x = 'Dmin', data = sample)


data is distribited equally

drops on every minute of hour is similar

In [224]:
# Perform the countplot on the DDweek data
sns.countplot(x = 'DDweek', data = sample)


drop offs on every day of a week are almost similar

In [225]:
# Perform the countplot on the Temp data
sns.countplot(x = 'Temp', data = sample)


the distribution of data is skewed toward right

trips are avoided in less temperature

In [226]:
# Perform the countplot on the Precip data
sns.countplot(x = 'Precip', data = sample)


the distribution of data is skewed towards left

no trips are planned during rainy or snowy wheather

In [227]:
# Perform the countplot on the wind data
sns.countplot(x = 'Wind', data = sample)


the distribution of data is skewed towards left

trips are mostly avoided in windy wheather

In [228]:
# Perform the countplot on the himid hour data
sns.countplot(x = 'Humid', data = sample)


people mostly avoid to go on trips on less humid or in extra humid wheather

In [229]:
# Perform the countplot on the solar data
sns.countplot(x = 'Solar', data = sample)


the data distribution is skewed towards left side

In [230]:
# Perform the countplot on the snow data
sns.countplot(x = 'Snow', data = sample)


the data distribution is skewed towards left side

more trips are during  when there is no snowfall.

In [231]:
# Perform the countplot on the GroundTemp data
sns.countplot(x = 'GroundTemp', data = sample)


the distribution of data is multimodel

In [232]:
# Perform the countplot on the dust data
sns.countplot(x = 'Dust', data = sample)


distribution of data is skewed towards left

more trips are done when there is less dust

## Boxen Plot

The boxen plot, otherwise known as a Letter-value plot, is a box plot meant for large data sets (n > 10,000).

The Boxen plot is very similar to box plot, except for the fact that it plots different quartile values.

By plotting different quartile values, we are able to understand the shape of the distribution particularly in the head end and tail end.

In [233]:
# Perform boxen plot between distance and Duration 
sns.boxenplot(x = 'Distance', y = 'Duration', data = sample)

distribution of distance is increasing as the duration  is increasing

In [234]:
# Perform boxen plot between Duration and Plong 
sns.boxenplot(x = 'Duration', y = 'PLong', data = sample)

from above boxen plot:

The distribution between lower adjacent value and upper adjacent value is symmetrical.
There is no relation between Plong and  Duration

In [235]:
# Perform boxen plot between PLatd and Duration 
sns.boxenplot(x = 'PLatd', y = 'Duration', data = sample)

from above boxen plot:

The distribution between lower adjacent value and upper adjacent value is symmetrical.
There is no relation between duration and Platd



In [236]:
# Perform boxen plot between DLong and Duration 
sns.boxenplot(x = 'DLong', y = 'Duration', data = sample)

from above boxen plot:

The distribution between lower adjacent value and upper adjacent value is symmetrical.
There is no relation between duration and Dlong

In [237]:
# Perform boxen plot between Dlatd and Duration 
sns.boxenplot(x = 'DLatd', y = 'Duration', data = sample)

from above boxen plot:

The distribution between lower adjacent value and upper adjacent value is symmetrical. There is no relation between duration and Dlatd

In [238]:
# Perform boxen plot between Pday and Duration 
sns.boxenplot(x = 'Pday', y = 'Duration', data = sample)

from above boxen plot:

The distribution between lower adjacent value and upper adjacent value is symmetrical. There is no relation between duration and pday

In [239]:
# Perform boxen plot between Phour and Duration 
sns.boxenplot(x = 'Phour', y = 'Duration', data = sample)

there is weak relation between duration and Phours

In [240]:
# Perform boxen plot between Pmin and Duration 
sns.boxenplot(x = 'Pmin', y = 'Duration', data = sample)

from above boxen plot:

The distribution between lower adjacent value and upper adjacent value is symmetrical. There is no relation between duration and Pmin

In [241]:
# Perform boxen plot between Dday and Duration 
sns.boxenplot(x = 'Dday', y = 'Duration', data = sample)

from above boxen plot:

The distribution between lower adjacent value and upper adjacent value is symmetrical. There is no relation between duration and Dday

In [242]:
# Perform boxen plot between Dhour and Duration 
sns.boxenplot(x = 'Dhour', y = 'Duration', data = sample)

there is weak relation between duration and Dhour

In [243]:
# Perform boxen plot between Dmin and Duration 
sns.boxenplot(x = 'Dmin', y = 'Duration', data = sample)

there is no relation between duration and Dmin

In [244]:
# Perform boxen plot between Dday and Duration 
sns.boxenplot(x = 'Dday', y = 'Duration', data = sample)

from above boxen plot:

The distribution between lower adjacent value and upper adjacent value is symmetrical. There is no relation between duration and Dday

In [245]:
# Perform boxen plot between temp and Duration 
sns.boxenplot(x = 'Temp', y = 'Duration', data = sample)

there is positive relation between duration and temperature

In [246]:
# Perform boxen plot between precip and Duration 
sns.boxenplot(x = 'Precip', y = 'Duration', data = sample)

no relation between duration and precipitation

In [247]:
# Perform boxen plot between wind and Duration 
sns.boxenplot(x = 'Wind', y = 'Duration', data = sample)

from above boxen plot:

The distribution between lower adjacent value and upper adjacent value is symmetrical. There is no relation between duration and wind

In [248]:
# Perform boxen plot between humid and Duration 
sns.boxenplot(x = 'Humid', y = 'Duration', data = sample)

duration and humidity have weak correlation between them

In [250]:
# Perform boxen plot between solar and Duration 
plt.figure(figsize = (15, 15))
sns.boxenplot(x = 'Solar', y = 'Duration', data = sample)

solar raditation and duration are correlated to eachother.

as the solar radiation increases duration also increases

In [251]:
# Perform boxen plot between snow and Duration 
sns.boxenplot(x = 'Snow', y = 'Duration', data = sample)

no relation between snow and duration

In [252]:
# Perform boxen plot between GroundTemp and Duration 
sns.boxenplot(x = 'GroundTemp', y = 'Duration', data = sample)

duration and ground temperature are correlated

In [253]:
# Perform boxen plot between dust and Duration 
sns.boxenplot(x = 'Dust', y = 'Duration', data = sample)

there is not much difference in the value of dust as duration increases.

there is weak relation between duration and dust.

In [254]:
# Perform boxen plot between distance and haversine 
sns.boxenplot(x = 'Distance', y = 'Haversine', data = sample)

there is weak relation between distance and haversine

In [255]:
# Perform boxen plot between distance and Pmonth 
sns.boxenplot(x = 'Distance', y = 'Pmonth', data = sample)

there is no relation between distance and Pmonth

In [256]:
# Perform boxen plot between distance and Pday 
sns.boxenplot(x = 'Distance', y = 'Pday', data = sample)

no relation between distance Pday

In [257]:
# Perform boxen plot between distance and Pmin 
sns.boxenplot(x = 'Distance', y = 'Pmin', data = sample)

there is no relation between distance and Pmin

In [258]:
# Perform boxen plot between distance and PDweek 
sns.boxenplot(x = 'Distance', y = 'PDweek', data = sample)

there is no relation between distance and PDweek

In [259]:
# Perform boxen plot between distance and Dmonth 
sns.boxenplot(x = 'Distance', y = 'Dmonth', data = sample)

there is no relation bertween distance and Dmonth

In [260]:
# Perform boxen plot between distance and Dday 
sns.boxenplot(x = 'Distance', y = 'Dday', data = sample)

there is no relation between distance and Dday

## DENDOGRAM

The dendrogram is a visual representation of the compound correlation data. The individual compounds are arranged along the bottom of the dendrogram and referred to as leaf nodes. Compound clusters are formed by joining individual compounds or existing compound clusters with the join point referred to as a node.

In [ ]:
# Plot a Dendrogram on the columns of the dataset
import plotly.figure_factory as ff
fig = ff.create_dendrogram(sample)
fig.update_layout(width=800, height=500)
fig.show()


Observation: All connected are compoundly related to each other like latitudes to longitudes, distance to duration and many more.

## Violin Plot

1. A violin plot is a method of plotting numeric data.

1. Violin plots are similar to box plots, except that they also show the probability density of the data at different values, usually smoothed by a kernel density estimator.

3. It has:

    1. Median (a white dot on the violin plot)
    2. Interquartile range (the black bar in the center of violin)
    3. The lower/upper adjacent values (the black lines stretched from the bar) — defined as first quartile — 1.5 IQR and third quartile + 1.5 IQR respectively.

In [ ]:
# Perform violin plot between distance and Duration 
sns.violinplot(x = 'Distance', y = 'Duration', data = sample)

Distance and duration arecorrelated

as duration increases distance also increases

In [ ]:
# Perform violin plot between PLong and Duration 
sns.violinplot(x = 'PLong', y = 'Duration', data = sample)

from above violin plot:

The distribution between lower adjacent value and upper adjacent value is symmetrical.
There is no relation between duration and PLong 

In [ ]:
# Perform violin plot between Platd and Duration 
sns.violinplot(x = 'PLatd', y = 'Duration', data = sample)

from above violin plot:

The distribution between lower adjacent value and upper adjacent value is symmetrical.

There is no relation between duration and PLatd 

In [ ]:
# Perform violin plot between Dlong and Duration 
sns.violinplot(x = 'DLong', y = 'Duration', data = sample)

Dlong distribution for all values of duration is simmilar.

there is no relation between DLong and duration.

In [ ]:
# Perform violin plot between DLatd and Duration 
sns.violinplot(x = 'DLatd', y = 'Duration', data = sample)

from above violin plot:

The distribution between lower adjacent value and upper adjacent value is symmetrical.
There is no relation between duration and DLatd 

In [ ]:
# Perform violin plot between Pmin and Duration 
sns.violinplot(x = 'Pmin', y = 'Duration', data = sample)

from above violin plot:

The distribution between lower adjacent value and upper adjacent value is symmetrical. 

There is no relation between duration and Pmin

In [ ]:
# Perform violin plot between Dhour and Duration 
sns.violinplot(x = 'Dhour', y = 'Duration', data = sample)

from above violin plot:

The distribution between lower adjacent value and upper adjacent value is symmetrical. 

There is no relation between duration and Dday

In [ ]:
# Perform violin plot between Dmin and Duration 
sns.violinplot(x = 'Dmin', y = 'Duration', data = sample)

Dmin distribution for all values of Duration is similar.

In [ ]:
# Perform violin plot between Precip and Duration 
sns.violinplot(x = 'Precip', y = 'Duration', data = sample)

precip distribution for all values of duration is simmilar.

In [ ]:
# Perform violin plot between wind and Duration 
sns.violinplot(x = 'Wind', y = 'Duration', data = sample)

wind distribution for all values of duration is simmilar.


In [ ]:
# Perform violin plot between humid and Duration 
sns.violinplot(x = 'Humid', y = 'Duration', data = sample)

as duration increases there is slight change in the humidity 

humidity is decreasing as duration is increasing

In [ ]:
# Perform violin plot between solar and Duration 
sns.violinplot(x = 'Solar', y = 'Duration', data = sample)

solar radiation and duration have positive correlation

as value of solar radiation increases duration also increases.

In [ ]:
# Perform violin plot between GroundTemp and Duration 
sns.violinplot(x = 'GroundTemp', y = 'Duration', data = sample)

as the value of duration increases value of ground temperature also slightly increases 

there is weak correlation between ground temperature and duration.

In [ ]:
# Perform violin plot between dust and Duration 
sns.violinplot(x = 'Dust', y = 'Duration', data = sample)

from above violin plot:

The distribution between lower adjacent value and upper adjacent value is symmetrical. 

There is weak relation between duration and dust

In [ ]:
# Perform violin plot between distance and Plong 
sns.violinplot(x = 'Distance', y = 'PLong', data = sample)

there is no relation between PLong and Distance

In [ ]:
# Perform violin plot between distance and PLatd 
sns.violinplot(x = 'Distance', y = 'PLatd', data = sample)

there is no relation between PLatd and Distance

In [ ]:
# Perform violin plot between distance and Dlong 
sns.violinplot(x = 'Distance', y = 'DLong', data = sample)

there is no relation between DLatd and Distance

In [ ]:
# Perform violin plot between distance and Dlatd 
sns.violinplot(x = 'Distance', y = 'DLatd', data = sample)

there is no relation between DLatd and Distance

In [ ]:
# Perform violin plot between distance and Pday
sns.violinplot(x = 'Distance', y = 'Pday', data = sample)

there is no relation between Pday and Distance

## Modelling

NOTE:
The dataset has 9.6 million samples.

We have used only 100000 samples for training.

If you want you can use complete dataset.

Using complete dataset will take longer time to train the model.

In [ ]:
# Splitting data into Labels and target
def proc_df(df, y_fld=None):
    df = df.copy()
    if y_fld is None: y = None
    else:
        y = df[y_fld].values
    res = [df, y]
    return res   

In [ ]:
# Processing the whole dataframe
X, y = proc_df(sample, 'Duration')

In [ ]:
# Splitting the dataset into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [ ]:
# Check the shape of all the splitted dataset
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


In [ ]:
# importing necessary libraries for geting metrics of models
import math
import sklearn.metrics as metrics
from sklearn.metrics import median_absolute_error

# Function for calculating RMSE 
def rmse(x,y): 
    return math.sqrt(((x-y)**2).mean())

# Function for calculating all the relevant metrics 
def print_score(m):
    res = [rmse(m.predict(X_train), Y_train),rmse(m.predict(X_test), Y_test),
           m.score(X_train, Y_train),m.score(X_test, Y_test),
           median_absolute_error(m.predict(X_train), Y_train),median_absolute_error(m.predict(X_test), Y_test),
           metrics.mean_absolute_error(m.predict(X_train), Y_train),metrics.mean_absolute_error(m.predict(X_test), Y_test),
          
          ]
    

    print("RMSE-Train: " + str(res[0]) + "\nRMSE-Test: " + str(res[1]) + "\nScore-Train: " + str(res[2]) + "\nScore-Test: " + str(res[3]) +
         "\nMedAE-Train: " + str(res[4]) + "\nMedAE-Test: " + str(res[5]) + "\nMeanAE-Train: " + str(res[6]) + "\nMeanAE-Test: " + str(res[7]))

In [ ]:
# Visualize importance of all the features in the dataset for the prediction

def visualize_importance(feature_importances, feat_train_df):
    feature_importance_df = pd.DataFrame()
     # creating dataframe for feature name and feature importance
    _df = pd.DataFrame()
    _df['feature_importance'] = feature_importances
    _df['column'] = feat_train_df.columns
    feature_importance_df = pd.concat([feature_importance_df, _df], 
                                      axis=0, ignore_index=True)
    # grouping all data and sorting in descending order
    order = feature_importance_df.groupby('column')\
        .sum()[['feature_importance']]\
        .sort_values('feature_importance', ascending=False).index[:50]
    # ploting feature importance data using boxenplot
    fig, ax = plt.subplots(figsize=(8, max(6, len(order) * .25)))
    sns.boxenplot(data=feature_importance_df, 
                  x='feature_importance', 
                  y='column', 
                  order=order, 
                  ax=ax, 
                  palette='viridis', 
                  orient='h')
    ax.tick_params(axis='x', rotation=0)
    ax.set_title('Importance')
    ax.grid()
    fig.tight_layout()
    # return fig, ax
    return fig, ax


# Linear Regression

Linear regression attempts to model the relationship between two variables by fitting a linear equation to observed data. One variable is considered to be an explanatory variable, and the other is considered to be a dependent variable.

In [ ]:
%%time
# Fit a Linear Regression model to the train dataset

# Import LinearRegressor
from sklearn.linear_model import LinearRegressor
# Instantiate the model
lm = LinearRegressor()
# Fit the model to the data
lm.fit(X_train, y_train)
# print score of the model
print_score(lm)
# visualizing the inportance of features.
visualize_importance(lm.coef_, X_train)

## Random Forest Regressor

Random forest is a flexible, easy to use machine learning algorithm that produces, even without hyper-parameter tuning, a great result most of the time. It is also one of the most used algorithms, because of its simplicity and diversity.

In [ ]:
%%time
# Fit a Random Forest Regressor model to the train dataset

# Import RandomForrestRegressor
from sklearn.ensemble import RandomForestRegressor
# Instantiate the model
rf = RandomForestRegressor()
# Fit the model to the data
rf.fit(X_train, y_train)
# print score of the model
print_score(rf)
# visualizing the importance of features.
visualize_importance(rf.feature_importances_, X_train)

## KNeighbors Regressor

KNN regression is a non-parametric method that, in an intuitive manner, approximates the association between independent variables and the continuous outcome by averaging the observations in the same neighbourhood. The size of the neighbourhood needs to be set by the analyst or can be chosen using cross-validation to select the size that minimises the mean-squared error.

In [ ]:
%%time
# Fit a K-Neighbour Regressor model to the train dataset

# Import KNeighbourRegressor
from sklearn.neighbors import KNeighborsRegressor
# Instantiate the model
knn = KNeighborsRegressor()
# Fit the model to the data
knn.fit(X_train, y_train)
# print score of the model
print_score(knn)

## Gradient Boosting Regressor

Gradient Boosting Algorithm is generally used when we want to decrease the Bias error. it builds an additive model in a forward stage-wise fashion; it allows for the optimization of arbitrary differentiable loss functions. In each stage a regression tree is fit on the negative gradient of the given loss function.

In [ ]:
%%time

# Fit a Gradient Boosting Regressor model to the train dataset

# Import GradientBoostingRegressor
from sklearn.ensemble GradientBoostingRegressor
# Instantiate the model
gb = GradientBoostingRegressor()
# Fit the model to the data
gb.fit(X_train, y_train)
# print score of the model
print_score(gb)
# visualizing the inportance of features.
visualize_importance(gb.feature_importances_, X_train)

## Decision Tree Regressor

Decision tree builds regression or classification models in the form of a tree structure. It breaks down a dataset into smaller and smaller subsets while at the same time an associated decision tree is incrementally developed. The final result is a tree with decision nodes and leaf nodes.

In [ ]:
%%time
# Fit a Decision Tree Regressor model to the train dataset

# Import DecisionTreeRegressor
from sklearn.tree DecisionTreeRegressor
# Instantiate the model
dtree = DecisionTreeRegressor
# Fit the model to the data
dtree.fit(X_train, y_train)
# print score of the model
print_score(dtree)
# visualizing the inportance of features.
visualize_importance(dtree.feature_importances_, X_train)

## AdaBoostRegressor

An AdaBoost regressor is a meta-estimator that begins by fitting a regressor on the original dataset and then fits additional copies of the regressor on the same dataset but where the weights of instances are adjusted according to the error of the current prediction.

In [ ]:
%%time
# Fit a AdaBoost Regressor model to the train dataset

# Import AdaBoostRegressor
from sklearn.ensemble import AdaBoostRegressor
# Instantiate the model
ab = AdaBoostRegressor()
# Fit the model to the data
ab.fit(X_train, y_train)
# print score of the model
print_score(ab)
# visualizing the inportance of features.
visualize_importance(ab.feature_importances_, X_train)

## XGB Regressor

XGBoost is an ensemble learning method. Sometimes, it may not be sufficient to rely upon the results of just one machine learning model. Ensemble learning offers a systematic solution to combine the predictive power of multiple learners. The resultant is a single model which gives the aggregated output from several models.

In [ ]:
%%time
# Import XGBRegressor
import xgboost as xgb
# Instantiate the model
xgb = xgb.XGBRegressor()
# Fit the model to the data
xgb.fit(X_train, y_train)
# print score of the model
print_score(xgb)
# visualizing the inportance of features.
visualize_importance(xgb.feature_importances_, X_train)

## Light Gradient Boosted Machine

In [ ]:
%%time
# import LGBMregressor
import lightgbm as lgbm
# Instantiate the model
lgbm = lgbm.LGBMRegressor()
# Fit the model to the data
lgbm.fit(X_train, y_train)
# print score of the model
print_score(lgbm)
# visualizing the inportance of features.
visualize_importance(lgbm.feature_importances_, X_train)

## Comparing all the model based on metric

In [ ]:
# Helper function for comparing models metrics
def compare_models(models,names,X_train,y_train,X_test,y_test):
  # the libraries we need
  import sklearn.metrics as metrics
  from sklearn.model_selection import train_test_split


  # now, create a list with the objects 
  data = {'Metric':['rmse','MedAE','MAE','R-squared']}
  df_train = pd.DataFrame(data)
  df_test = pd.DataFrame(data)

  def rmse(x,y):
    return math.sqrt(((x-y)**2).mean())


  for (model,name) in zip(models,names):
    y_pred= model.predict(X_test) # then predict on the test set
    res = [rmse(model.predict(X_train), y_train),rmse(model.predict(X_test), y_test),
              metrics.median_absolute_error(model.predict(X_train), y_train),metrics.median_absolute_error(model.predict(X_test), y_test),
              metrics.mean_absolute_error(model.predict(X_train), y_train),metrics.mean_absolute_error(model.predict(X_test), y_test),
              metrics.r2_score(model.predict(X_train), y_train),metrics.r2_score(model.predict(X_test), y_test)]
    # get metrics of each model, and add to dataframe 
  df_train[name] = [res[0], res[2], res[4], res[6]]
  df_test[name] = [res[1], res[3], res[5], res[7]]
  return df_train,df_test   

In [ ]:
# list of models object
models = [lm, rf, knn, gb, dtree, ab, xgb, lgbm]
# list of models name
names = ['Linear Regression', 'Random Forest Tree', 'Gradient Boosting', 'Decision Tree', 'AdaBoost', 'XGBoost', 'Light Gradient Boosting Machine']
# use function for comparing models by passing list of models object, names, train and test data
df_train, df_test = compare_models(models, names, X_train, y_train, X_test, y_test)

### RMSE of all models on train and test data

In [ ]:
# printing rmse comparision of model on train and test
print(df_train['AdaBoost'])
print(df_test['AdaBoost'])

### All metrics on train and test data

In [ ]:
# printing comparision of model on train and test
print("on Train")
print(df_train)
print("on test")
print(df_test)

## Hyperparameter Tunning

A hyperparameter is a parameter whose value is set before the learning process begins.

Hyperparameters tuning is crucial as they control the overall behavior of a machine learning model.

Every machine learning models will have different hyperparameters that can be set.

## RamdomizedSearchCV

RandomizedSearchCV is very useful when we have many parameters to try and the training time is very long.

1. The first step is to write the parameters that we want to consider
2. From these parameters select the best ones.(which are printed in output)

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X[:5000],y[:5000],test_size=0.25,random_state=123)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
# Use the random grid to search for best hyperparameters
def random_Search(model,X_train, Y_train,param_grid):
  from sklearn.model_selection import RandomizedSearchCV

  # Random search of parameters, using 3 fold cross validation, 
  # search across 100 different combinations, and use all available cores
  random = RandomizedSearchCV(estimator = model, param_distributions = param_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
  random.fit(X_train, Y_train)
  print("\n Best parameters: ",random.best_params_)

In [ ]:
%%time
# create parameters dict in list for tunning
param_grid = {'n_estimators': 400, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features':
'auto', 'max_depth': 60, 'bootstrap': True}
# passing data for hyper parameter tunning with RandomSearchCV
random_Search(rf, X_train, Y_train, param_grid)

In [ ]:
%%time
# create parameters dict in list for tunning
param_grid = {'p': 1, 'n_neighbors': 29, 'leaf_size': 19}
# passing data for hyper parameter tunning with RandomSearchCV
random_Search(knn, X_train, Y_train, param_grid)

In [ ]:
%%time
# create parameters dict in list for tunning
param_grid = {'n_estimators': 100, 'learning_rate': 0.01}
# passing data for hyper parameter tunning with RandomSearchCV
random_Search(ab, X_train, Y_train, param_grid)

In [ ]:
%%time
# create parameters dict in list for tunning
param_grid = {'min_child_weight': 1, 'max_depth': 5, 'learning_rate': 0.15, 'gamma': 0.0,
'colsample_bytree': 0.4}
# passing data for hyper parameter tunning with RandomSearchCV
random_Search(xgb, X_train, Y_train, param_grid)

In [ ]:
%%time
# create parameters dict in list for tunning
param_grid = {'n_estimators': 100, 'min_samples_split': 5, 'max_depth': 5, 'learning_rate': 0.2}
# passing data for hyper parameter tunning with RandomSearchCV
random_Search(lgbm, X_train, Y_train, param_grid)

## Using best hyperparameters

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.25,random_state=123)
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

## RandomForest Model

In [ ]:
%%time
# Fit a Random Forest Regressor model to the train dataset

# Instantiate the model
rf = RandomForestRegressor(n_estimators = 400, min_samples_split = 5, min_samples_leaf = 2, max_features =
'auto', max_depth = 60, bootstrap = True)
# Fit the model to the data
rf.fit(X_train, Y_train)
# print score of the model
print_score(rf)
# visualizing the importance of features.
visualize_importance(rf.feature_importances_, X_train)

## KNN Regressor

In [ ]:
%%time
# Fit a K-Neighbour Regressor model to the train dataset

# Instantiate the model
knn = KNeighborsRegressor(p = 1, n_neighbors = 29, leaf_size = 19)
# Fit the model to the data
knn.fit(X_train, Y_train)
# print score of the model
print_score(knn)

## Adaboost Regressor

In [ ]:
%%time
# Fit a AdaBoost Regressor model to the train dataset
# Instantiate the model
ab = AdaBoostRegressor(n_estimators = 100, learning_rate = 0.01)
# Fit the model to the data
ab.fit(X_train, Y_train)
# print score of the model
print_score(ab)
# visualizing the inportance of features.
visualize_importance(ab.feature_importances_, X_train)

## XGBoost Regressor

In [ ]:
%%time

# Instantiate the model
xgb = xgb.XGBRegressor(min_child_weight= 1, max_depth= 5, learning_rate= 0.15, gamma= 0.0,
colsample_bytree= 0.4)
# Fit the model to the data
xgb.fit(X_train, Y_train)
# print score of the model
print_score(xgb)
# visualizing the inportance of features.
visualize_importance(xgb.feature_importances_, X_train)

In [ ]:
%%time

# Instantiate the model
lgbm = lgbm.LGBMRegressor(n_estimators= 100, min_samples_split = 5, max_depth =5, learning_rate = 0.2)
# Fit the model to the data
lgbm.fit(X_train, Y_train)
# print score of the model
print_score(lgbm)
# visualizing the inportance of features.
visualize_importance(lgbm.feature_importances_, X_train)

In [ ]:
# list of models object
models = [lm, rf, knn, gb, dtree, ab, xgb, lgbm]
# list of models name
names = ['Linear Regression', 'Random Forest Tree', 'Gradient Boosting', 'Decision Tree', 'AdaBoost', 'XGBoost', 'Light Gradient Boosting Machine']
# use function for comparing models by passing list of models object, names, train and test data
df_train, df_test = compare_models(models, names, X_train, Y_train, X_test, Y_test)

In [ ]:
# printing comparision of model on train and test
print("Train")
print(df_train)
print("Test")
print(df_test)

## CONCLUSION

In this project we have done an extensive Exploratory Data Analysis, visualized the data with amazing plots, build different models, visualized feature importance, did hyper parameter tunning of each model. Finally we found out that, from all the columns in the data Distance is the most important feature of them all and after Hyperparameter Tuning Adaboost Regressor performed much better when compared to remaining models.


<center><h1>Congratulations for completing the assignment.</h1></center>
<br/>
<center>You have learned a lot while doing this assignment.</center>